In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

In [2]:
import os
import io
import pandas as pd
import tensorflow.compat.v1 as tf

In [3]:
from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

In [11]:
DEFAULT_CSV_INPUT = 'default_csv_input.csv'
DEFAULT_IMAGE_DIR = 'default_image_dir'
DEFAULT_OUTPUT_PATH = 'default_output_tfrecord.tfrecord'

In [4]:
# Defines flags to specify input csv path, output tfrecord file path and 
# directory containing images
# Allow users to specify input/output paths when running the script from command line.

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('image_dir', '', 'Path to images')
FLAGS = flags.FLAGS

In [5]:
# Converts class labels (text) into corresponding integer IDs. 
def class_text_to_int(row_label):
    if row_label == 'Nyatapola':
        return 1
    elif row_label == 'Golden Gate':
        return 2
    else:
        return None

In [6]:
# Splits the dataset(DataFrame) into groups based on a specified key
# Prepares the data for processing by grouping data related to the same image. 
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [7]:
# Creates a tensorflow example protocol buffer message from a group of data
# (annotations for one image)
# reads the image file, encodes it, and extracts image properties such as 
# width and height. 
# constructs bounding boxes and class labels for objects detected in the image. 
# Finally creates a TF Example message containing image data, annotations and 
# metadata. 

def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


In [12]:
def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path or DEFAULT_OUTPUT_PATH)
    path = os.path.join(FLAGS.image_dir or DEFAULT_IMAGE_DIR)
    examples = pd.read_csv(FLAGS.csv_input or DEFAULT_CSV_INPUT)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


In [13]:
if __name__ == '__main__':
    tf.app.run()


FileNotFoundError: [Errno 2] No such file or directory: 'default_csv_input.csv'